## 🎓 Distillation: GPT-2 → H-AKORN H-LLM

Transferir conhecimento do GPT-2 pré-treinado para o modelo hiperbólico H-AKORN.

**Teacher**: GPT-2 (124M params, Euclidean)
**Student**: H-AKORN H-LLM (~30M params, Hyperbolic + Kuramoto)

**Distillation loss**:
```
L_distill = α·KL(student || teacher) + (1-α)·L_LM + λ_hidden·L_hidden
```

Onde:
- KL divergence alinha distribuições de output
- L_LM é a standard language modeling loss
- L_hidden alinha representações escondidas (Euclidean → Hyperbolic)

## 🔁 Fluxo de Treinamento do H-LLM

```text
Modelo inicial (H-LLM random / pré-treino)
        ↓
[ FASE 1 ] Destilação (Teacher ON)
        ↓
[ FASE 2 ] LM-only (Teacher OFF)
        ↓
[ FASE 3 ] Avaliação / uso / refinamento



In [ ]:
# @title Deletar checkpoints antigos
#!rm -rf /content/outputs/h_llm/checkpoints/*
print("✅ Checkpoints deletados")

✅ Checkpoints deletados


In [ ]:
# @title 1.1 Upload CGT Project with H-AKORN
from google.colab import files
import zipfile
import os

!rm -rf /content/cgt_project_with_hakorn
print('📤 Upload cgt_project_with_hakorn.zip:')

uploaded = files.upload()
for f in uploaded:
    if f.endswith('.zip'):
        with zipfile.ZipFile(f, 'r') as z:
            z.extractall('/content')
        print(f'✅ Extracted: {f}')
        os.remove(f)

# Verificar H-AKORN
required = [
    '/content/cgt_project_with_hakorn/src/cgt/geometry/lorentz_hardened.py',
    '/content/cgt_project_with_hakorn/src/cgt/models/hakorn/model.py',
    '/content/cgt_project_with_hakorn/src/cgt/models/hakorn/phase_dynamics.py',
    '/content/cgt_project_with_hakorn/src/cgt/models/hakorn/attention.py',
]

print('\n📋 Checking H-AKORN modules:')
for p in required:
    print(f"{'✅' if os.path.exists(p) else '❌ MISSING:'} {p}")

📤 Upload cgt_project_with_hakorn.zip:


Saving cgt_project_with_hakorn.zip to cgt_project_with_hakorn.zip
✅ Extracted: cgt_project_with_hakorn.zip

📋 Checking H-AKORN modules:
✅ /content/cgt_project_with_hakorn/src/cgt/geometry/lorentz_hardened.py
✅ /content/cgt_project_with_hakorn/src/cgt/models/hakorn/model.py
✅ /content/cgt_project_with_hakorn/src/cgt/models/hakorn/phase_dynamics.py
✅ /content/cgt_project_with_hakorn/src/cgt/models/hakorn/attention.py


In [ ]:
# @title 1.2 Install Dependencies
!pip install -q transformers datasets tqdm POT torch
# @title 0. Fix PyTorch (executar primeiro)
import importlib
import sys

# Limpar módulos torch parcialmente carregados
mods_to_remove = [k for k in sys.modules if 'torch' in k]
for m in mods_to_remove:
    del sys.modules[m]

print("✅ Módulos limpos. Continue com as próximas células.")

✅ Módulos limpos. Continue com as próximas células.


In [ ]:
# @title 2. Setup & Imports
import sys
sys.path.insert(0, '/content/cgt_project_with_hakorn/src')

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from dataclasses import dataclass, asdict
import json
from pathlib import Path

# H-AKORN imports
from cgt.geometry.lorentz_hardened import LorentzSubstrateHardened
from cgt.models.hakorn import HAKORNTransformer, HAKORNLoss
from cgt.losses.hyperbolic_lm_losses import (
    HyperbolicLMLoss,
    TeacherDistillationLoss,
    RadiusRegularization,
    ManifoldFidelityLoss,
)

# GPT-2 Tokenizer
from transformers import GPT2Tokenizer
from datasets import load_dataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'🔧 Device: {device}')
if torch.cuda.is_available():
    print(f'   GPU: {torch.cuda.get_device_name(0)}')
    print(f'   VRAM: {torch.cuda.get_device_properties(0).total_memory/1e9:.1f} GB')

print(f"PyTorch: {torch.__version__}")
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {DEVICE}")


OUTPUT_DIR = Path('/content/outputs/h_llm')
CHECKPOINT_DIR = OUTPUT_DIR / 'checkpoints'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)

print('\n✅ H-AKORN modules imported!')

🔧 Device: cuda
   GPU: NVIDIA A100-SXM4-80GB
   VRAM: 85.2 GB
PyTorch: 2.9.0+cu126
Device: cuda

✅ H-AKORN modules imported!


In [ ]:
# @title 3. Load GPT-2 Tokenizer
print("🔄 Loading GPT-2 tokenizer...")
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

print(f"✅ Tokenizer loaded")
print(f"   Vocab size: {tokenizer.vocab_size}")
print(f"   EOS token: {tokenizer.eos_token} (id={tokenizer.eos_token_id})")
print(f"   PAD token: {tokenizer.pad_token} (id={tokenizer.pad_token_id})")

# Test
test_text = "Hello, I am a hyperbolic language model!"
tokens = tokenizer.encode(test_text)
decoded = tokenizer.decode(tokens)
print(f"\n   Test: '{test_text}'")
print(f"   Tokens: {tokens}")
print(f"   Decoded: '{decoded}'")

🔄 Loading GPT-2 tokenizer...
✅ Tokenizer loaded
   Vocab size: 50257
   EOS token: <|endoftext|> (id=50256)
   PAD token: <|endoftext|> (id=50256)

   Test: 'Hello, I am a hyperbolic language model!'
   Tokens: [15496, 11, 314, 716, 257, 8718, 65, 4160, 3303, 2746, 0]
   Decoded: 'Hello, I am a hyperbolic language model!'


In [ ]:
# @title 4. Configuration
@dataclass
class TrainingConfig:
    # Model Architecture
    vocab_size: int = 50257       # GPT-2 vocab
    d_model: int = 512            # Embedding dimension
    num_layers: int = 6           # Transformer layers
    num_heads: int = 8            # Attention heads
    d_ff: int = 2048              # FFN dimension
    max_seq_len: int = 128        # Context window
    dropout: float = 0.1

    # H-AKORN Specific
    curvature: float = -1.0              # Hyperbolic curvature (K < 0)
    coupling_strength: float = 1.5       # Kuramoto coupling strength
    use_phase_modulation: bool = True    # Phase modulation in attention

    # Data
    seq_length: int = 128

    # Loss Weights
    lambda_sync: float = 0.1             # Synchronization loss
    lambda_variance: float = 0.05        # Phase variance loss
    lambda_infonce: float = 0.3          # InfoNCE contrastive
    lambda_manifold: float = 0.5         # Manifold fidelity
    lambda_radius: float = 0.001         # Radius regularization

    # Training
    batch_size: int = 16
    learning_rate: float = 1e-4
    num_epochs: int = 10
    warmup_steps: int = 500
    gradient_clip: float = 1.0
    early_stop_patience: int = 5

    # Checkpointing
    output_dir: str = '/content/outputs/h_akorn_llm'
    checkpoint_every: int = 500
    eval_every: int = 100

config = TrainingConfig()

print("⚙️ H-AKORN H-LLM Configuration:")
print(f"   Model: d={config.d_model}, L={config.num_layers}, H={config.num_heads}")
print(f"   H-AKORN: coupling={config.coupling_strength}, κ={config.curvature}")
print(f"   Phase modulation: {config.use_phase_modulation}")
print(f"   Training: batch={config.batch_size}, lr={config.learning_rate}, epochs={config.num_epochs}")
print(f"   Output: {config.output_dir}")

⚙️ H-AKORN H-LLM Configuration:
   Model: d=512, L=6, H=8
   H-AKORN: coupling=1.5, κ=-1.0
   Phase modulation: True
   Training: batch=16, lr=0.0001, epochs=10
   Output: /content/outputs/h_akorn_llm


In [ ]:
# @title 5. Load Dataset (ROBUSTO, SEM HF DEPENDENCY)
print("🔄 Loading WikiText-2...")

from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader

# Try Hugging Face WikiText (may fail due to HF issue)
try:
    dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
    print("✅ WikiText-2 loaded from Hugging Face")
    use_synthetic = False

except Exception as e:
    print("⚠️ Failed to load WikiText-2 from Hugging Face")
    print(f"   Reason: {type(e).__name__}: {e}")
    print("⚠️ Falling back to synthetic dataset")
    use_synthetic = True


class WikiTextDataset(Dataset):
    def __init__(self, texts, tokenizer, seq_length: int):
        self.seq_length = seq_length

        all_text = " ".join(t for t in texts if t and t.strip())
        enc = tokenizer(
            all_text,
            return_attention_mask=False,
            return_token_type_ids=False,
        )

        self.tokens = enc["input_ids"]
        self.num_chunks = len(self.tokens) // seq_length
        self.tokens = self.tokens[: self.num_chunks * seq_length]

    def __len__(self):
        return self.num_chunks

    def __getitem__(self, idx):
        start = idx * self.seq_length
        end = start + self.seq_length
        tokens = torch.tensor(self.tokens[start:end], dtype=torch.long)
        return {"input_ids": tokens, "labels": tokens}


class SyntheticDataset(Dataset):
    def __init__(self, vocab_size: int, seq_length: int, n_samples: int = 100_000):
        self.vocab_size = vocab_size
        self.seq_length = seq_length
        self.n_samples = n_samples

    def __len__(self):
        return self.n_samples

    def __getitem__(self, idx):
        tokens = torch.randint(
            0,
            self.vocab_size,
            (self.seq_length,),
            dtype=torch.long,
        )
        return {"input_ids": tokens, "labels": tokens}


# Build datasets
if not use_synthetic:
    print("📘 Using WikiText-2")

    train_dataset = WikiTextDataset(
        dataset["train"]["text"],
        tokenizer,
        config.max_seq_len,   # ✅ CORRIGIDO: max_seq_len
    )

    val_dataset = WikiTextDataset(
        dataset["validation"]["text"],
        tokenizer,
        config.max_seq_len,   # ✅ CORRIGIDO: max_seq_len
    )

else:
    print("🧪 Using Synthetic Dataset")

    train_dataset = SyntheticDataset(
        vocab_size=tokenizer.vocab_size,
        seq_length=config.max_seq_len,   # ✅ CORRIGIDO: max_seq_len
        n_samples=100_000,
    )

    val_dataset = SyntheticDataset(
        vocab_size=tokenizer.vocab_size,
        seq_length=config.max_seq_len,   # ✅ CORRIGIDO: max_seq_len
        n_samples=10_000,
    )


train_dataloader = DataLoader(  # ✅ CORRIGIDO: train_dataloader (não train_loader)
    train_dataset,
    batch_size=config.batch_size,
    shuffle=True,
    drop_last=True,
)

val_dataloader = DataLoader(  # ✅ CORRIGIDO: val_dataloader (não val_loader)
    val_dataset,
    batch_size=config.batch_size,
    drop_last=True,
)

print("✅ DataLoaders ready")
print(f"   Train batches: {len(train_dataloader)}")
print(f"   Val batches: {len(val_dataloader)}")

🔄 Loading WikiText-2...
⚠️ Failed to load WikiText-2 from Hugging Face
   Reason: HfHubHTTPError: 404 Client Error: Not Found for url: https://huggingface.co/api/datasets/wikitext/revision/b08601e04326c79dfdd32d625aee71d232d685c3 (Request ID: Root=1-697969e5-7af5e3a5412750d4619dea5a;06491b38-93fb-4dfd-8107-24ebf7c03eb7)

Sorry, we can't find the page you are looking for.
⚠️ Falling back to synthetic dataset
🧪 Using Synthetic Dataset
✅ DataLoaders ready
   Train batches: 6250
   Val batches: 625


In [ ]:
# @title 6. Create H-AKORN H-LLM Model (RE-RUN)
from cgt.geometry.lorentz_hardened import LorentzSubstrateHardened, LorentzConfig

print("🔄 Creating H-AKORN Hyperbolic LLM...")

# Create LorentzConfig
lorentz_config = LorentzConfig(
    intrinsic_dim=config.d_model,
    initial_curvature=abs(1.0 / config.curvature),
    learnable_curvature=False,
    eps=1e-6,
)

# Create substrate
substrate = LorentzSubstrateHardened(config=lorentz_config)

# Add distance() method wrapper
def distance_wrapper(self, x, y):
    return self.dist(x, y)
LorentzSubstrateHardened.distance = distance_wrapper

print(f"   Substrate: dim={lorentz_config.intrinsic_dim}, K={lorentz_config.initial_curvature:.1f}")

# Create H-AKORN Transformer (com coupling.py CORRIGIDO)
model = HAKORNTransformer(
    vocab_size=config.vocab_size,
    d_model=config.d_model,
    num_layers=config.num_layers,
    num_heads=config.num_heads,
    d_ff=config.d_ff,
    dropout=config.dropout,
    substrate=substrate,
    coupling_strength=config.coupling_strength,
    use_phase_modulation=config.use_phase_modulation,
).to(device)

# Create losses
hakorn_loss = HAKORNLoss(
    lambda_sync=config.lambda_sync,
    lambda_variance=config.lambda_variance,
)

hyperbolic_loss = HyperbolicLMLoss(
    substrate=substrate,
    vocab_size=config.vocab_size,
    lambda_lm=1.0,
    lambda_infonce=config.lambda_infonce,
    lambda_f1=config.lambda_manifold,
    lambda_radius=config.lambda_radius,
)

optimizer = AdamW(model.parameters(), lr=config.learning_rate)

print(f"✅ H-AKORN H-LLM created (with FIXED coupling.py)")
print(f"   Parameters: {model.get_num_params():,}")

🔄 Creating H-AKORN Hyperbolic LLM...
   Substrate: dim=512, K=1.0
✅ H-AKORN H-LLM created (with FIXED coupling.py)
   Parameters: 18,916,808


In [ ]:
# @title 7. H-AKORN Trainer Class
class HAKORNLLMTrainer:
    """H-AKORN H-LLM Trainer with Kuramoto dynamics, checkpointing, early stopping."""

    def __init__(self, model, config, tokenizer):
        self.model = model
        self.config = config
        self.tokenizer = tokenizer
        self.device = next(model.parameters()).device

        self.optimizer = AdamW(model.parameters(), lr=config.learning_rate)

        # Criar diretórios
        Path(config.output_dir).mkdir(parents=True, exist_ok=True)
        Path(f"{config.output_dir}/checkpoints").mkdir(exist_ok=True)

        self.global_step = 0
        self.best_val_loss = float('inf')
        self.patience_counter = 0

    def train_step(self, batch):
        """Single training step with H-AKORN losses."""
        self.model.train()
        self.optimizer.zero_grad()

        input_ids = batch['input_ids'].to(self.device)
        labels = batch['input_ids'].to(self.device)

        # Forward (H-AKORN returns: logits, loss, order_params, hidden_states)
        output = self.model(input_ids, labels=labels, return_dict=True)

        # H-AKORN loss (synchronization + variance)
        hakorn_loss_dict = hakorn_loss(
            output['loss'],
            output['all_order_params'],
        )

        # Hyperbolic loss (manifold + radius + InfoNCE)
        hyp_loss_dict = hyperbolic_loss(
            output['logits'],
            labels,
            output['hidden_states'],
            return_components=True,
        )

        # Combined loss
        total_loss = hakorn_loss_dict['total'] + hyp_loss_dict['total']

        # Backward
        total_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.config.gradient_clip)
        self.optimizer.step()

        self.global_step += 1

        # Metrics
        metrics = {
            'loss': total_loss.item(),
            'lm_loss': output['loss'].item(),
            'order_param': output['all_order_params'][-1].mean().item(),
            'sync_loss': hakorn_loss_dict.get('sync_loss', 0),
            'var_loss': hakorn_loss_dict.get('var_loss', 0),
        }

        return metrics

    @torch.no_grad()
    def eval_step(self, dataloader):
        """Evaluation on validation set."""
        self.model.eval()
        total_loss = 0
        total_order = 0
        total_steps = 0

        for batch in dataloader:
            input_ids = batch['input_ids'].to(self.device)
            output = self.model(input_ids, labels=input_ids, return_dict=True)

            total_loss += output['loss'].item()
            total_order += output['all_order_params'][-1].mean().item()
            total_steps += 1

        return {
            'val_loss': total_loss / total_steps,
            'val_order': total_order / total_steps,
        }

    def save_checkpoint(self, val_loss):
        """Save checkpoint."""
        checkpoint = {
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'global_step': self.global_step,
            'val_loss': val_loss,
            'config': self.config,
        }

        path = f"{self.config.output_dir}/checkpoints/checkpoint_step{self.global_step}.pt"
        torch.save(checkpoint, path)

        # Save best
        if val_loss < self.best_val_loss:
            self.best_val_loss = val_loss
            best_path = f"{self.config.output_dir}/best_model.pt"
            torch.save(checkpoint, best_path)
            print(f"💾 New best! val_loss={val_loss:.4f}")
            self.patience_counter = 0
        else:
            self.patience_counter += 1

    def train(self, train_loader, val_loader, num_epochs):
        """Full training loop with early stopping."""
        train_losses = []
        val_losses = []
        order_params = []

        print(f"🚀 Training H-AKORN H-LLM for {num_epochs} epochs...")

        for epoch in range(num_epochs):
            print(f"\n📍 Epoch {epoch+1}/{num_epochs}")

            # Training
            epoch_metrics = []
            pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}")

            for batch in pbar:
                metrics = self.train_step(batch)
                epoch_metrics.append(metrics)

                pbar.set_postfix({
                    'loss': f"{metrics['loss']:.3f}",
                    'r': f"{metrics['order_param']:.2f}",
                })

                # Eval
                if self.global_step % self.config.eval_every == 0:
                    val_metrics = self.eval_step(val_loader)
                    print(f"\n✓ Step {self.global_step}: val_loss={val_metrics['val_loss']:.4f}, r={val_metrics['val_order']:.3f}")

                    # Checkpoint
                    if self.global_step % self.config.checkpoint_every == 0:
                        self.save_checkpoint(val_metrics['val_loss'])
                        val_losses.append((self.global_step, val_metrics['val_loss']))

            # Epoch summary
            avg_loss = np.mean([m['loss'] for m in epoch_metrics])
            avg_order = np.mean([m['order_param'] for m in epoch_metrics])
            train_losses.append(avg_loss)
            order_params.append(avg_order)

            print(f"📊 Epoch {epoch+1}: loss={avg_loss:.4f}, order={avg_order:.3f}")

            # Early stopping
            if self.patience_counter >= self.config.early_stop_patience:
                print(f"\n⚠️ Early stopping (patience={self.config.early_stop_patience})")
                break

        return train_losses, val_losses, order_params

print("✅ H-AKORN Trainer class defined")

✅ H-AKORN Trainer class defined


In [ ]:
# @title 8. Create Trainer & Train
# trainer = HAKORNLLMTrainer(model, config, tokenizer)

# print("🚀 H-AKORN H-LLM Trainer created")
# print(f"   Model: {model.get_num_params():,} parameters")
# print(f"   Dataset: WikiText-2")
# print(f"   Epochs: {config.num_epochs}")
# print(f"   Learning rate: {config.learning_rate}")
# print(f"   Kuramoto coupling: {config.coupling_strength}")
# print(f"   Output: {config.output_dir}")
# print()

# # Start training
# train_hist, val_hist, order_hist = trainer.train(
#     train_dataloader,
#     val_dataloader,
#     num_epochs=config.num_epochs,
# )

# print("\n✅ Training complete!")

In [ ]:
# @title DEBUG: Check tensor shapes
# print("🔍 Debugging tensor shapes...")

# # Create dummy input
# dummy_input = torch.randint(0, config.vocab_size, (2, 128)).to(device)

# try:
#     with torch.no_grad():
#         output = model(dummy_input)
#     print("✅ Forward pass OK!")
# except RuntimeError as e:
#     print(f"❌ Error: {e}")

#     # Print model structure
#     print("\n📋 Model config:")
#     print(f"   d_model: {config.d_model}")
#     print(f"   num_heads: {config.num_heads}")
#     print(f"   batch_size: {config.batch_size}")
#     print(f"   max_seq_len: {config.max_seq_len}")

In [ ]:
# @title 9. Train (Early Stopping decides when to stop)
#train_hist, val_hist = trainer.train(train_loader, val_loader)

In [ ]:
# @title 10. Training Analysis Plots
def plot_analysis(th, vh, path):
    fig, ax = plt.subplots(2, 3, figsize=(18, 10))
    fig.suptitle('H-LLM Training Analysis (GPT-2 Tokenizer)', fontsize=14, fontweight='bold')

    s = [h['step'] for h in th]
    loss = [h['loss'] for h in th]
    ppl = [h['ppl'] for h in th]
    r = [h['radius_mean'] for h in th]
    rs = [h['radius_std'] for h in th]
    viol = [h['mean_violation'] for h in th]
    lr = [h['lr'] for h in th]
    vs = [h['step'] for h in vh]
    vl = [h['val_loss'] for h in vh]
    vp = [h['val_ppl'] for h in vh]

    # Loss
    ax[0,0].plot(s, loss, alpha=0.3, label='Train')
    w = min(50, len(loss)//10) if len(loss) > 10 else 1
    if w > 1:
        ax[0,0].plot(s[w-1:], np.convolve(loss, np.ones(w)/w, 'valid'), lw=2, label=f'MA{w}')
    ax[0,0].plot(vs, vl, 'ro-', ms=4, label='Val')
    ax[0,0].set_xlabel('Step'); ax[0,0].set_ylabel('Loss')
    ax[0,0].set_title('Loss'); ax[0,0].legend(); ax[0,0].grid(True, alpha=0.3)

    # Perplexity
    ax[0,1].plot(vs, vp, 'g^-', ms=6)
    ax[0,1].set_xlabel('Step'); ax[0,1].set_ylabel('PPL')
    ax[0,1].set_title('Validation Perplexity'); ax[0,1].grid(True, alpha=0.3)

    # Radius
    ax[0,2].plot(s, r, label='Mean')
    ax[0,2].fill_between(s, [x-y for x,y in zip(r,rs)], [x+y for x,y in zip(r,rs)], alpha=0.3)
    ax[0,2].axhline(5.0, color='r', ls='--', label='R_max')
    ax[0,2].set_xlabel('Step'); ax[0,2].set_ylabel('Radius')
    ax[0,2].set_title('Embedding Radius'); ax[0,2].legend(); ax[0,2].grid(True, alpha=0.3)

    # Manifold fidelity
    ax[1,0].semilogy(s, viol)
    ax[1,0].axhline(1e-5, color='g', ls='--', label='Target')
    ax[1,0].set_xlabel('Step'); ax[1,0].set_ylabel('Violation')
    ax[1,0].set_title('Manifold Fidelity'); ax[1,0].legend(); ax[1,0].grid(True, alpha=0.3)

    # Learning Rate
    ax[1,1].plot(s, lr)
    ax[1,1].set_xlabel('Step'); ax[1,1].set_ylabel('LR')
    ax[1,1].set_title('Learning Rate'); ax[1,1].grid(True, alpha=0.3)

    # Train PPL
    ax[1,2].semilogy(s, ppl, alpha=0.3)
    if w > 1:
        ax[1,2].semilogy(s[w-1:], np.convolve(ppl, np.ones(w)/w, 'valid'), lw=2)
    ax[1,2].set_xlabel('Step'); ax[1,2].set_ylabel('PPL')
    ax[1,2].set_title('Train Perplexity'); ax[1,2].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(path, dpi=150, bbox_inches='tight')
    plt.show()
    print(f"📊 Saved: {path}")

if 'train_hist' in dir() and train_hist:
    plot_analysis(train_hist, val_hist, OUTPUT_DIR / 'training_analysis.png')
else:
    print("⚠️ Nenhum histórico de treino. Execute a célula de treinamento primeiro.")

⚠️ Nenhum histórico de treino. Execute a célula de treinamento primeiro.


## 🎓 Distillation: GPT-2 → H-LLM

Transferir conhecimento do GPT-2 pré-treinado para o modelo hiperbólico.

**Benefícios:**
- Converge muito mais rápido
- Aprende distribuições mais suaves
- Melhor generalização

In [ ]:
# @title 11. Load GPT-2 Teacher

import torch
from transformers import GPT2LMHeadModel

# IMPORTS CORRETOS — sempre pelo pacote
from cgt.distillation import (
    GPT2TeacherWrapper,
    DistillationConfig,
    DistillationTrainer,
    plot_distillation_analysis,
)

print("🔄 Loading GPT-2 Teacher...")
teacher = GPT2TeacherWrapper(model_name="gpt2", device=DEVICE)

# Teste rápido do teacher
with torch.no_grad():
    test_ids = torch.randint(0, 50257, (1, 32), device=DEVICE)
    t_out = teacher(test_ids)
    print(f"✅ Teacher output shape: {t_out['logits'].shape}")
    print(f"   Hidden shape: {t_out['hidden_states'].shape}")

🔄 Loading GPT-2 Teacher...
✅ Teacher output shape: torch.Size([1, 32, 50257])
   Hidden shape: torch.Size([1, 32, 768])


In [ ]:
# @title 12. Distillation Configuration
distill_config = DistillationConfig(
    alpha=0.5,
    temperature=2.0,
    lambda_distill=0.7,

    learning_rate=5e-5,
    max_steps=1000,          # ⬅️ teto realista
    warmup_steps=300,        # ⬅️ fase útil da distil

    early_stopping_patience=1,  # ⬅️ 1 eval sem melhora = STOP
    eval_every=200,
    log_every=100,
)


print("Distillation Config:")
for k, v in asdict(distill_config).items():
    print(f"  {k}: {v}")

Distillation Config:
  alpha: 0.5
  temperature: 2.0
  lambda_distill: 0.7
  learning_rate: 5e-05
  weight_decay: 0.01
  max_steps: 1000
  warmup_steps: 300
  gradient_clip: 1.0
  early_stopping_patience: 1
  early_stopping_min_delta: 0.01
  checkpoint_every: 1000
  eval_every: 200
  log_every: 100
  keep_last_n_checkpoints: 3


In [ ]:
# @title 13. Create Distillation Trainer & Resume
distill_trainer = DistillationTrainer(
    student=model,
    teacher=teacher,
    config=distill_config,
    tokenizer=tokenizer,
    checkpoint_dir=CHECKPOINT_DIR,
    device=DEVICE
)

# Check for migrated checkpoint first, then old checkpoint
migrated_ckpt = CHECKPOINT_DIR / "distill_latest_migrated.pt"
old_ckpt = CHECKPOINT_DIR / "distill_latest.pt"

if migrated_ckpt.exists():
    print(f"✅ Found migrated checkpoint: {migrated_ckpt}")
    resume = input("Resume from migrated checkpoint? (y/n): ").strip().lower()
    if resume == "y":
        distill_trainer.load(str(migrated_ckpt))
        print(f"✅ Loaded migrated checkpoint at step {distill_trainer.step}")

        # Fix max_steps if needed
        if distill_trainer.step >= distill_config.max_steps:
            new_max_steps = max(20000, distill_trainer.step + 1000)
            print(f"\n⚠️  Step {distill_trainer.step} >= max_steps {distill_config.max_steps}")
            print(f"✅ Auto-fixing: max_steps → {new_max_steps}")
            distill_config.max_steps = new_max_steps
            distill_trainer.config.max_steps = new_max_steps

        print(f"   Remaining steps: {distill_config.max_steps - distill_trainer.step}")
    else:
        print("Starting fresh distillation")

elif old_ckpt.exists():
    print(f"⚠️  Found OLD checkpoint (needs migration): {old_ckpt}")
    migrate = input("Migrate now? (y/n): ").strip().lower()

    if migrate == "y":
        print("\n🔄 Migrating checkpoint...")
        import torch
        ckpt = torch.load(old_ckpt, map_location='cpu')
        old_state = ckpt['model']
        new_state = {}
        migrated_count = 0

        for k, v in old_state.items():
            if '.norm1.weight' in k:
                new_state[k.replace('.norm1.weight', '.norm1.layer_norm.weight')] = v
                migrated_count += 1
            elif '.norm1.bias' in k:
                new_state[k.replace('.norm1.bias', '.norm1.layer_norm.bias')] = v
                migrated_count += 1
            elif '.norm2.weight' in k:
                new_state[k.replace('.norm2.weight', '.norm2.layer_norm.weight')] = v
                migrated_count += 1
            elif '.norm2.bias' in k:
                new_state[k.replace('.norm2.bias', '.norm2.layer_norm.bias')] = v
                migrated_count += 1
            elif 'final_norm.weight' in k:
                new_state[k.replace('final_norm.weight', 'final_norm.layer_norm.weight')] = v
                migrated_count += 1
            elif 'final_norm.bias' in k:
                new_state[k.replace('final_norm.bias', 'final_norm.layer_norm.bias')] = v
                migrated_count += 1
            else:
                new_state[k] = v

        ckpt['model'] = new_state
        torch.save(ckpt, migrated_ckpt)
        print(f"✅ Migrated {migrated_count} keys → {migrated_ckpt}")

        # Load migrated checkpoint
        distill_trainer.load(str(migrated_ckpt))
        print(f"✅ Loaded migrated checkpoint at step {distill_trainer.step}")

        # Fix max_steps if needed
        if distill_trainer.step >= distill_config.max_steps:
            new_max_steps = max(20000, distill_trainer.step + 1000)
            print(f"\n⚠️  Step {distill_trainer.step} >= max_steps {distill_config.max_steps}")
            print(f"✅ Auto-fixing: max_steps → {new_max_steps}")
            distill_config.max_steps = new_max_steps
            distill_trainer.config.max_steps = new_max_steps

        print(f"   Remaining steps: {distill_config.max_steps - distill_trainer.step}")
    else:
        print("Starting fresh distillation")

else:
    print("Starting fresh distillation (no checkpoint found)")

# Final validation
print("\n" + "="*60)
print("📊 TRAINER STATUS")
print("="*60)
print(f"   Current step: {distill_trainer.step}")
print(f"   Max steps: {distill_config.max_steps}")
print(f"   Remaining: {distill_config.max_steps - distill_trainer.step}")
print(f"   Device: {DEVICE}")
print("="*60)

Starting fresh distillation (no checkpoint found)

📊 TRAINER STATUS
   Current step: 0
   Max steps: 1000
   Remaining: 1000
   Device: cuda


In [ ]:
# @title 🚨 EMERGENCY MEMORY FIX - Run NOW

import torch
import gc

print("=" * 70)
print("🚨 EMERGENCY MEMORY CLEANUP")
print("=" * 70)

# 1. AGGRESSIVE CACHE CLEARING
torch.cuda.empty_cache()
gc.collect()
torch.cuda.synchronize()
print("✅ Cache cleared")

# 2. CHECK MEMORY STATUS
if torch.cuda.is_available():
    total = torch.cuda.get_device_properties(0).total_memory / 1e9
    allocated = torch.cuda.memory_allocated() / 1e9
    reserved = torch.cuda.memory_reserved() / 1e9
    free = (torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated()) / 1e9

    print(f"\n📊 GPU Memory:")
    print(f"   Total: {total:.1f} GB")
    print(f"   Allocated: {allocated:.1f} GB ({allocated/total*100:.1f}%)")
    print(f"   Reserved: {reserved:.1f} GB")
    print(f"   Free: {free:.1f} GB ({free/total*100:.1f}%)")

    if allocated / total > 0.95:
        print(f"\n❌ CRITICAL: GPU is {allocated/total*100:.1f}% FULL!")
        print(f"   You MUST reduce batch size or model size")
    elif free < 1.0:
        print(f"\n⚠️  WARNING: Only {free:.1f} GB free")
        print(f"   Recommend batch_size=1")
    else:
        print(f"\n✅ OK: {free:.1f} GB available")

# 3. SET MINIMAL BATCH SIZE
BATCH_SIZE = 1  # CRITICAL: Must be 1
print(f"\n⚡ Setting BATCH_SIZE = {BATCH_SIZE}")

# 4. RECREATE DATALOADERS
from torch.utils.data import DataLoader

if 'collate_fn' not in globals():
    collate_fn = None

if 'train_dataset' in globals() and 'val_dataset' in globals():
    train_dataloader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        collate_fn=collate_fn,
        drop_last=True,
        num_workers=0,
    )

    val_dataloader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        collate_fn=collate_fn,
        drop_last=False,
        num_workers=0,
    )

    print(f"✅ Dataloaders recreated with batch_size={BATCH_SIZE}")
    print(f"   Train: {len(train_dataloader)} batches")
    print(f"   Val: {len(val_dataloader)} batches")
else:
    print("⚠️  Datasets not found - create them first")

# 5. REDUCE LM_HEAD CHUNK SIZES
if 'model' in globals():
    if hasattr(model, 'lm_head') and hasattr(model.lm_head, 'chunk_size'):
        model.lm_head.chunk_size = 25  # VERY SMALL
        if hasattr(model.lm_head, 'seq_chunk_size'):
            model.lm_head.seq_chunk_size = 16  # VERY SMALL
        print(f"✅ Reduced lm_head chunk_size to {model.lm_head.chunk_size}")
        if hasattr(model.lm_head, 'seq_chunk_size'):
            print(f"✅ Reduced seq_chunk_size to {model.lm_head.seq_chunk_size}")
    else:
        print("⚠️  lm_head doesn't have adjustable chunk_size")
else:
    print("⚠️  Model not found")

# 6. UPDATE CONFIG
if 'distill_config' in globals():
    distill_config.batch_size = BATCH_SIZE
    print(f"✅ Updated distill_config.batch_size = {BATCH_SIZE}")

print("\n" + "=" * 70)
print("⚠️  CRITICAL INSTRUCTIONS:")
print("=" * 70)
print("1. GPU is nearly FULL - batch_size MUST be 1")
print("2. Training will be VERY SLOW with batch_size=1")
print("3. Consider:")
print("   • Reduce model size (d_model, num_layers)")
print("   • Reduce sequence length (max_position_embeddings)")
print("   • Remove teacher model if using distillation")
print("=" * 70)

🚨 EMERGENCY MEMORY CLEANUP
✅ Cache cleared

📊 GPU Memory:
   Total: 85.2 GB
   Allocated: 0.7 GB (0.8%)
   Reserved: 84.6 GB
   Free: 84.4 GB (99.2%)

✅ OK: 84.4 GB available

⚡ Setting BATCH_SIZE = 1
✅ Dataloaders recreated with batch_size=1
   Train: 100000 batches
   Val: 10000 batches
✅ Reduced lm_head chunk_size to 25
✅ Reduced seq_chunk_size to 16
✅ Updated distill_config.batch_size = 1

⚠️  CRITICAL INSTRUCTIONS:
1. GPU is nearly FULL - batch_size MUST be 1
2. Training will be VERY SLOW with batch_size=1
3. Consider:
   • Reduce model size (d_model, num_layers)
   • Reduce sequence length (max_position_embeddings)
   • Remove teacher model if using distillation


In [ ]:
# @title 14. Run Distillation (Early Stopping decides)
distill_train_hist, distill_val_hist = distill_trainer.train(train_dataloader, val_dataloader)

In [ ]:
# @title 15. LM-only Configuration (POST-DISTILL)

from dataclasses import asdict

lm_config = DistillationConfig(
    alpha=0.0,
    lambda_distill=0.0,
    temperature=1.0,

    learning_rate=3e-5,
    max_steps=3000,
    warmup_steps=300,

    early_stopping_patience=2,
    eval_every=300,
    log_every=100,
)

print("LM-only config DEFINIDO:")
for k, v in asdict(lm_config).items():
    print(f"  {k}: {v}")

In [ ]:
# @title 16. LM-only Checkpoint Directory

from pathlib import Path

lm_checkpoint_dir = Path("/content/drive/MyDrive/H-LLM_Checkpoints/LM_ONLY")
lm_checkpoint_dir.mkdir(parents=True, exist_ok=True)

print("LM-only checkpoints:", lm_checkpoint_dir)

In [ ]:
# @title 17. Trainer LM-only (NO TEACHER)

lm_trainer = DistillationTrainer(
    student=model,
    teacher=None,          # NÃO PASSA TEACHER
    config=lm_config,
    tokenizer=tokenizer,
    checkpoint_dir=lm_checkpoint_dir,
    device=device,
)

In [ ]:
# @title 18. Run LM-only Training

lm_train_hist, lm_val_hist = lm_trainer.train(
    train_dataloader,
    val_dataloader
)

In [ ]:
# @title 15. Distillation Analysis Plots
if distill_train_hist:
    plot_distillation_analysis(
        distill_train_hist,
        distill_val_hist,
        OUTPUT_DIR / "distillation_analysis.png"
    )

In [ ]:
# @title 16. Load Best Distilled Model
import math
distill_best = CHECKPOINT_DIR / "distill_best.pt"
if distill_best.exists():
    ckpt = torch.load(distill_best, map_location=DEVICE)
    model.load_state_dict(ckpt["model"])
    print(f"✅ Best distilled model loaded from step {ckpt['step']}")
    print(f"   Val loss: {ckpt['best_val']:.4f}")
    print(f"   Val PPL: {math.exp(min(ckpt['best_val'], 20)):.1f}")
else:
    # Fallback to regular best
    best = CHECKPOINT_DIR / "best.pt"
    if best.exists():
        ckpt = torch.load(best, map_location=DEVICE)
        model.load_state_dict(ckpt["model"])
        print(f"✅ Best model loaded from step {ckpt['step']}")

model.eval()
print("\n🔮 Model ready for chat!")

In [ ]:
# @title 17. 🗣️ Chat Interativo com H-LLM (GERAÇÃO CORRETA)
"""


MUDANÇAS:
- Usa model.forward() para obter logits (consistente com treino)
- Valida se lm_head é hiperbólico ou euclidiano
- Remove tentativas de reconstruir logits manualmente
"""

import torch
import torch.nn.functional as F
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from datetime import datetime
import json

# ============================================================================
# CHAT CLASS
# ============================================================================
class HyperbolicChat:
    def __init__(self, model, tokenizer, device):
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
        self.conversation = []
        self.max_context = 512

        # Validar modelo
        self._validate_model()

    def _validate_model(self):
        """Valida componentes hiperbólicos do modelo."""
        # Verificar substrate
        has_substrate = hasattr(self.model, 'substrate') and self.model.substrate is not None

        # Verificar lm_head
        has_lm_head = hasattr(self.model, 'lm_head')

        if not has_substrate:
            print("⚠️  AVISO: Modelo sem substrate - usando geração Euclidiana")
            self.use_hyperbolic = False
            return

        if not has_lm_head:
            raise RuntimeError(
                "ERRO CRÍTICO: Modelo tem substrate mas não tem lm_head."
            )

        # Verificar tipo de lm_head
        lm_head = self.model.lm_head
        is_hyperbolic_head = (
            hasattr(lm_head, 'substrate') or
            hasattr(lm_head, 'compute_logits_hyperbolic') or
            'Hyperbolic' in type(lm_head).__name__
        )

        if not is_hyperbolic_head:
            print(f"⚠️  AVISO: lm_head ({type(lm_head).__name__}) é Euclidiano")
        else:
            print(f"✅ Geometria Hiperbólica VALIDADA")
            print(f"   Substrate: {type(self.model.substrate).__name__}")
            print(f"   LM Head: {type(lm_head).__name__}")

        self.use_hyperbolic = True

    def add_message(self, role, content):
        self.conversation.append({
            'role': role,
            'content': content,
            'time': datetime.now().strftime('%H:%M:%S')
        })

    @torch.no_grad()
    def generate(self, prompt, max_tokens=100, temperature=0.8, top_k=50, top_p=0.95):
        """
        Geração autoregressiva usando model.forward().

        IMPORTANTE: Usa EXATAMENTE o mesmo caminho do treino.
        """
        self.model.eval()

        # Construir contexto da conversa
        context = ""
        for msg in self.conversation:
            prefix = "User: " if msg['role'] == 'user' else "Assistant: "
            context += prefix + msg['content'] + "\n"
        context += "User: " + prompt + "\nAssistant:"

        # Tokenizar
        input_ids = self.tokenizer.encode(context, return_tensors='pt').to(self.device)

        # Truncar se muito longo
        if input_ids.shape[1] > self.max_context:
            input_ids = input_ids[:, -self.max_context:]

        # Gerar tokens
        generated = []
        for _ in range(max_tokens):
            # Forward pass - MESMO CAMINHO DO TREINO
            outputs = self.model(input_ids, return_dict=True)

            # Logits vêm do lm_head (hiperbólico ou euclidiano)
            logits = outputs['logits'][:, -1, :]

            # Aplicar temperatura
            if temperature != 1.0:
                logits = logits / temperature

            # Top-k filtering
            if top_k > 0:
                indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
                logits[indices_to_remove] = float('-inf')

            # Top-p filtering
            if top_p < 1.0:
                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)
                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
                sorted_indices_to_remove[..., 0] = 0
                indices_to_remove = sorted_indices_to_remove.scatter(1, sorted_indices, sorted_indices_to_remove)
                logits[indices_to_remove] = float('-inf')

            # Sample
            probs = F.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)

            # Stop conditions
            if next_token.item() == self.tokenizer.eos_token_id:
                break

            generated.append(next_token.item())
            input_ids = torch.cat([input_ids, next_token], dim=1)

            # Truncar
            if input_ids.shape[1] > self.max_context:
                input_ids = input_ids[:, -self.max_context:]

            # Stop at newline
            decoded = self.tokenizer.decode([next_token.item()])
            if '\n' in decoded and len(generated) > 10:
                break

        response = self.tokenizer.decode(generated, skip_special_tokens=True).strip()
        return response

    def chat(self, user_input, **kwargs):
        self.add_message('user', user_input)
        response = self.generate(user_input, **kwargs)
        self.add_message('assistant', response)
        return response

    def clear(self):
        self.conversation = []

    def get_html(self):
        html = '''
        <style>
            .chat-box { max-height: 400px; overflow-y: auto; padding: 10px;
                        border: 1px solid #ddd; border-radius: 10px; background: #f5f5f5; }
            .msg { margin: 8px 0; padding: 10px 15px; border-radius: 15px; max-width: 85%; }
            .user { background: #007bff; color: white; margin-left: auto; text-align: right; }
            .assistant { background: #28a745; color: white; }
            .role { font-size: 0.75em; font-weight: bold; margin-bottom: 3px; }
            .time { font-size: 0.65em; opacity: 0.7; margin-top: 3px; }
            .stats { font-size: 0.8em; color: #666; padding: 5px; margin-top: 10px;
                     background: white; border-radius: 5px; }
        </style>
        <div class="chat-box" id="chat-box">
        '''
        for msg in self.conversation:
            cls = msg['role']
            icon = '👤' if cls == 'user' else '🔮'
            html += f'''
            <div class="msg {cls}">
                <div class="role">{icon} {cls.title()}</div>
                <div>{msg['content']}</div>
                <div class="time">{msg['time']}</div>
            </div>
            '''
        html += f'''
        <div class="stats">💬 {len(self.conversation)} messages | 🔮 H-LLM</div>
        </div>
        <script>document.getElementById('chat-box').scrollTop = document.getElementById('chat-box').scrollHeight;</script>
        '''
        return html

# ============================================================================
# INTERFACE
# ============================================================================
chat = HyperbolicChat(model, tokenizer, DEVICE)

output = widgets.Output()
input_box = widgets.Textarea(
    placeholder='Digite sua mensagem...',
    layout=widgets.Layout(width='100%', height='60px')
)

send_btn = widgets.Button(description='📤 Enviar', button_style='primary')
clear_btn = widgets.Button(description='🗑️ Limpar', button_style='warning')
save_btn = widgets.Button(description='💾 Salvar', button_style='info')

temp_slider = widgets.FloatSlider(value=0.8, min=0.1, max=2.0, step=0.1, description='Temperatura:')
tokens_slider = widgets.IntSlider(value=100, min=20, max=300, step=10, description='Max Tokens:')
topk_slider = widgets.IntSlider(value=50, min=0, max=100, step=10, description='Top-K:')
topp_slider = widgets.FloatSlider(value=0.95, min=0.5, max=1.0, step=0.05, description='Top-P:')

def update():
    with output:
        clear_output(wait=True)
        display(HTML(chat.get_html()))

def on_send(b):
    text = input_box.value.strip()
    if not text:
        return
    input_box.value = ''
    with output:
        clear_output(wait=True)
        display(HTML(chat.get_html() + '<p style="color:#888">🔄 Generating...</p>'))
    chat.chat(text, max_tokens=tokens_slider.value, temperature=temp_slider.value,
              top_k=topk_slider.value, top_p=topp_slider.value)
    update()

def on_clear(b):
    chat.clear()
    update()

def on_save(b):
    path = OUTPUT_DIR / f'chat_{datetime.now().strftime("%Y%m%d_%H%M%S")}.json'
    with open(path, 'w') as f:
        json.dump(chat.conversation, f, indent=2)
    print(f"💾 Saved: {path}")

send_btn.on_click(on_send)
clear_btn.on_click(on_clear)
save_btn.on_click(on_save)

header = widgets.HTML('''
<h2>🔮 Chat com H-LLM (Hyperbolic Language Model)</h2>
<p style="color:#666">Geração via forward() oficial - Consistente com treino</p>
''')

params_box = widgets.VBox([
    widgets.HTML('<b>⚙️ Parâmetros de Geração:</b>'),
    widgets.HBox([temp_slider, tokens_slider]),
    widgets.HBox([topk_slider, topp_slider]),
])

buttons_box = widgets.HBox([send_btn, clear_btn, save_btn])

display(header)
display(params_box)
display(output)
display(input_box)
display(buttons_box)

update()

print('''
╔════════════════════════════════════════════════════════════════╗
║  🔮 H-LLM Chat (CORREÇÃO DEFINITIVA)                           ║
╠════════════════════════════════════════════════════════════════╣
║  ✅ Geração usa model.forward() - MESMO CAMINHO DO TREINO     ║
║  ✅ Logits vêm do lm_head oficial                             ║
║  ✅ SEM reconstrução manual de distâncias                     ║
║  ✅ SEM try/except mascarando erros                           ║
╚════════════════════════════════════════════════════════════════╝
''')

In [ ]:
# @title 18. Save to Google Drive
from datetime import datetime

SAVE_TO_DRIVE = True  # @param {type:"boolean"}

if SAVE_TO_DRIVE:
    from google.colab import drive
    import shutil

    drive.mount('/content/drive')

    dp = Path('/content/drive/MyDrive/H-LLM_Checkpoints')
    dp.mkdir(parents=True, exist_ok=True)
    ts = datetime.now().strftime('%Y%m%d_%H%M%S')

    for f in ['best.pt', 'latest.pt']:
        src = CHECKPOINT_DIR / f
        if src.exists():
            shutil.copy(src, dp / f'{f.replace(".pt", "")}_{ts}.pt')
            print(f"✅ Saved {f}")

    plot = OUTPUT_DIR / 'training_analysis.png'
    if plot.exists():
        shutil.copy(plot, dp / f'analysis_{ts}.png')

    print(f"\n📁 Files saved to: {dp}")

Mounted at /content/drive

📁 Files saved to: /content/drive/MyDrive/H-LLM_Checkpoints


## 📋 Summary

### ✅ Features:
- **GPT-2 Tokenizer** (50257 tokens reais)
- **WikiText-2** dataset real
- **🎓 GPT-2 Distillation** - transferência de conhecimento
- **Checkpoints** automáticos
- **Early Stopping** inteligente
- **Chat Interativo** com parâmetros ajustáveis

### 🔧 Fluxo:
1. Células 1-10: Setup + Treino básico (opcional)
2. Células 11-16: **Distillation GPT-2 → H-LLM**
3. Célula 17: Chat interativo

### 💬 Chat Parâmetros:
- **Temperatura**: Criatividade (0.1=conservador, 2.0=criativo)
- **Max Tokens**: Tamanho máximo da resposta
- **Top-K**: Filtro de tokens mais prováveis
- **Top-P**: Nucleus sampling